In [ ]:
# 0. import libraries

import numpy as np
import keras
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator

base_dir = './out/'
prefix = 'vgg16'
n_train_samples = 2000
train_file_name = 'bottleneck_features_train.npy'

n_validation_samples = 800
validation_file_name = 'bottleneck_features_validation.npy'


In [ ]:
# 1-1.Create Bottleneck Features data from image data

# Load VGG16 model & weight
model = VGG16(include_top=False, weights='imagenet')
model.summary()

# convert image data into numpy array using imageDataGenerator
## training data
image_data_generator = ImageDataGenerator(rescale=1.0/255)
train_data = image_data_generator.flow_from_directory(
    'dataset/trainData',
    target_size=(150, 150),
    batch_size=32,
    class_mode=None,
    shuffle=False
)

## validation data 
image_data_generator = ImageDataGenerator(rescale=1.0/255)
validation_data = image_data_generator.flow_from_directory(
    'dataset/validationData',
    target_size=(150, 150),
    batch_size=32,
    class_mode=None,
    shuffle=False
)

# Generate bottleneck featuresata using VGG16
## training data
bottleneck_feature_train = model.predict_generator(train_data, n_train_samples, verbose=1)

## validation data 
bottleneck_feature_validation = model.predict_generator(validation_data, n_validation_samples, verbose=1)

# Save bottleneck features training data
## traning data
np.save(base_dir + prefix + train_file_name, bottleneck_feature_train)

## validation data
np.save(base_dir + prefix + validation_file_name, bottleneck_feature_validation)



In [ ]:
# 1-2. Load Existing Bottleneck features data 
train_data  = np.load(base_dir + prefix + train_file_name)
len_input_samples = len(train_data)
train_labels = np.array([0] * int(len_input_samples/2) + [1] * int(len_input_samples / 2))

validation_data = np.load(base_dir + prefix + validation_file_name)
validation_labels = np.array([0] * int(n_validation_samples / 2 *32) + [1] * int(n_validation_samples / 2 * 32))


In [ ]:
# 2. Create Model
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras import optimizers

input_shape = train_data.shape[1:]

model = Sequential()
model.add(Flatten(input_shape=input_shape))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer=optimizers.SGD(lr=1e-4, momentum=0.9), metrics=['accuracy'])

In [ ]:
# 3. Train Model 
result_dir =  './out/history_vgg16_transfer_learning.txt'

# callbacks
callbacks = keras.callbacks.TensorBoard(log_dir='./out/tensorBoard', histogram_freq=0)

# train model
history = model.fit(train_data, train_labels, epochs=20, batch_size=32, callbacks=[callbacks], validation_data=(validation_data, validation_labels))

# Save weight
model.save_weights('./out/vgg16_transferlearning_weights.h5')

# Save history
loss = history.history['loss']
acc = history.history['acc']
val_loss = history.history['val_loss']
val_acc = history.history['val_acc']

with open(result_dir, "w") as fp:
    fp.write("epoch\tloss\tacc\tval_loss\tval_acc\n")
    for i in range(len(acc)):
        fp.write("%d\t%f\t%f\t%f\t%f\n" % (i, loss[i], acc[i], val_loss[i], val_acc[i]))
